# 💵 ETL de Cotações — Projeto Web Unifor ETL

## 📊 Painel com os dados do Projeto:
https://app.powerbi.com/view?r=eyJrIjoiZTQ1MTM2NDktNjYwNy00ZWVkLWJmZTQtY2E1NTZkOTg3YWIwIiwidCI6IjkzYzkzMDI3LTY2MjctNDA0My04MDI4LWFkYzQxN2M1MTg1NyJ9

## 🎯 Objetivo
Este notebook implementa a coleta diária da **cotação do dólar comercial (USD/BRL)** e de outros indicadores de mercado, realizando as etapas de **extração, transformação e carga (ETL)** em um banco analítico **DuckDB**.  
O objetivo é consolidar uma série histórica diária de preços (abertura, fechamento, máxima e mínima), que poderá ser utilizada para análises econômicas e correlações com notícias coletadas em outras etapas do projeto.



## 🌐 Fonte de Dados
Os dados de cotação são obtidos de fontes públicas de mercado financeiro, utilizando **APIs e bibliotecas Python** de extração automatizada (como "yfinance' ou 'requests').

| Fonte | Descrição |
|--------|------------|
| **Yahoo Finance** | Cotações históricas do par 'USDBRL=X' (Dólar/Real) |
| **Banco Central do Brasil (BACEN)** *(opcional)* | Dados de referência oficial do câmbio PTAX |

As informações incluem variações diárias do dólar, que são estruturadas e persistidas localmente no banco DuckDB.



## ⚙️ Estrutura do Pipeline

1. **Coleta:**  
   - Extração automatizada de cotações via API ('yfinance').  
   - Período de coleta: últimos 6 meses até a data atual.  
   - Criação do DataFrame com colunas OHLC (Open, High, Low, Close).

2. **Transformação:**  
   - Conversão de datas ('DatetimeIndex' → 'DATE').  
   - Padronização de nomes de colunas ('abertura', 'fechamento', 'alta', 'baixa').  
   - Cálculo de colunas derivadas (ex: 'tendencia', 'filtroperiodo').  
   - Validação de dados ausentes e tipos corretos.

3. **Carga:**  
   - Salvamento dos dados no banco **DuckDB ('dados_dolar.duckdb')**.  
   - Atualização incremental com base na chave primária 'data'.  
   - Exportação adicional em '.csv' e '.parquet'.

---

## 🧱 Estrutura da Tabela 'dolar_diario'

| Coluna | Tipo | Descrição |
|--------|------|-----------|
| 'data' | DATE | Data de referência da cotação |
| 'abertura' | DOUBLE | Valor de abertura do dólar |
| 'fechamento' | DOUBLE | Valor de fechamento do dólar |
| 'alta' | DOUBLE | Valor máximo do dia |
| 'baixa' | DOUBLE | Valor mínimo do dia |
| 'tendencia' | TEXT | Indicador de alta/baixa baseado em variação diária |
| 'filtroperiodo' | TEXT | Marcador auxiliar para filtros de período |
| 'dataCarga' | TIMESTAMP | Data e hora da carga no banco |

---

## 🧾 Logs e Monitoramento
O notebook utiliza **logs estruturados** em console e arquivo ('logs/api_dolar_diario.log') para:
- Registrar início e fim de cada etapa do ETL.  
- Indicar contagem de registros coletados e duração da execução.  
- Reportar erros de conexão, schema ou duplicação de dados.  

Cada execução é registrada com timestamp, permitindo auditoria e acompanhamento histórico do pipeline.

---

## 🧮 Reprodutibilidade
- Notebook executável de ponta a ponta, com dependências listadas em 'requirements.txt'.  
- Banco analítico em **DuckDB**, atualizado incrementalmente.  
- Saídas auxiliares exportadas em '.csv' para inspeção.  
- Ambiente compatível com reexecução automatizada via 'papermill' ou 'nbconvert'.

---

📅 **Última atualização:** 25/10/2025  
👨‍💻 **Autor:** ANDERSON DE OLIVEIRA SILVA — Projeto Web Unifor ETL (2025)


In [8]:
import pandas as pd
import requests
from datetime import date, datetime
import time
import duckdb

# opcional: instalar yfinance se necessário
# !pip install yfinance
import yfinance as yf

In [9]:
# ---------------------------
# Helpers de LOG
# ---------------------------
def now():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

class Step:
    def __init__(self, name):
        self.name = name
        self.t0 = time.time()
        print(f"[{now()}] [{self.name}] INÍCIO")
    def ok(self, extra="OK"):
        dt = time.time() - self.t0
        print(f"[{now()}] [{self.name}] FIM — {extra} ({dt:.2f}s)")
    def fail(self, err):
        dt = time.time() - self.t0
        print(f"[{now()}] [{self.name}] FALHA — {err} ({dt:.2f}s)")

In [10]:

# =============================================================================
# 1) PTAX (BCB) — fechamento diário
# =============================================================================
s = Step("PTAX/BCB - Coleta")
try:
    ini = "04-01-2025"
    fim = date.today().strftime("%m-%d-%Y")
    url = (
      "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
      f"CotacaoDolarPeriodo(dataInicial=@ini,dataFinalCotacao=@fim)?"
      f"@ini='{ini}'&@fim='{fim}'&$select=cotacaoCompra,dataHoraCotacao&$top=10000&$format=json"
    )

    data = requests.get(url, timeout=30).json()["value"]
    df = pd.DataFrame(data)

    # total bruto (todas as cotações do dia)
    print(f"[{now()}] [PTAX] Registros brutos: {len(df)}")

    # última cotação do dia (fechamento)
    df["data"] = pd.to_datetime(df["dataHoraCotacao"]).dt.date
    df_daily = df.sort_values("dataHoraCotacao").groupby("data").tail(1)
    df_daily = df_daily[["data","cotacaoCompra"]].reset_index(drop=True)

    # formata data para dd/mm/yyyy (exibição)
    df_daily["data"] = pd.to_datetime(df_daily["data"]).dt.strftime("%d/%m/%Y")

    # janela de datas (convertendo para comparar corretamente)
    min_ptax = pd.to_datetime(df_daily["data"], dayfirst=True).min().date()
    max_ptax = pd.to_datetime(df_daily["data"], dayfirst=True).max().date()
    print(f"[{now()}] [PTAX] Fechamentos diários: {len(df_daily)} | Janela: {min_ptax} → {max_ptax}")

    s.ok()
except Exception as e:
    s.fail(e)
    raise

[2025-10-25 20:01:05] [PTAX/BCB - Coleta] INÍCIO
[2025-10-25 20:01:07] [PTAX] Registros brutos: 146
[2025-10-25 20:01:07] [PTAX] Fechamentos diários: 145 | Janela: 2025-04-01 → 2025-10-24
[2025-10-25 20:01:07] [PTAX/BCB - Coleta] FIM — OK (1.77s)


In [11]:
# =============================================================================
# 2) PTAX — carga no DuckDB
# =============================================================================
s = Step("PTAX/BCB - Carga DuckDB")
try:
    DB_PATH = "dados_dolar.duckdb"
    con = duckdb.connect(DB_PATH)

    con.execute("""
    CREATE TABLE IF NOT EXISTS dolar_diario (
      data DATE,
      cotacao_compra DOUBLE
    );
    """)

    con.register("df_stage", df_daily)
    con.execute("""
    MERGE INTO dolar_diario AS t
    USING (
      SELECT
        STRPTIME(data, '%d/%m/%Y')::DATE AS data,
        CAST(cotacaoCompra AS DOUBLE)     AS cotacao_compra
      FROM df_stage
    ) AS s
    ON t.data = s.data
    WHEN MATCHED THEN UPDATE SET
      cotacao_compra = s.cotacao_compra
    WHEN NOT MATCHED THEN INSERT (data, cotacao_compra)
    VALUES (s.data, s.cotacao_compra);
    """)

    total_ptax_db = con.execute("SELECT COUNT(*) FROM dolar_diario;").fetchone()[0]
    print(f"[{now()}] [PTAX] Linhas na tabela dolar_diario: {total_ptax_db}")

    con.execute("""
    COPY dolar_diario TO './exports/dolar_diario.csv' (HEADER, DELIMITER ',');
    """)
    print(f"[{now()}] [PTAX] Export CSV -> ./exports/dolar_diario.csv")

    con.unregister("df_stage")
    con.close()
    s.ok("Carga concluída")
except Exception as e:
    s.fail(e)
    raise

[2025-10-25 20:01:07] [PTAX/BCB - Carga DuckDB] INÍCIO
[2025-10-25 20:01:07] [PTAX] Linhas na tabela dolar_diario: 145
[2025-10-25 20:01:07] [PTAX] Export CSV -> ./exports/dolar_diario.csv
[2025-10-25 20:01:07] [PTAX/BCB - Carga DuckDB] FIM — Carga concluída (0.06s)


In [12]:
# =============================================================================
# 3) Yahoo Finance — OHLC diário USDBRL=X
# =============================================================================
s = Step("Yahoo OHLC - Coleta")
try:
    ANO = date.today().year
    inicio = pd.Timestamp(ANO, 4, 1)
    fim_desejado = pd.Timestamp(ANO, 10, 31)
    fim = min(fim_desejado, pd.Timestamp.today().normalize())

    ticker = "USDBRL=X"

    df_yf = yf.download(
      ticker,
      start=inicio,
      end=fim + pd.Timedelta(days=1),  # inclui o último dia
      interval="1d",
      auto_adjust=False,
      progress=False,
    )
    if df_yf.empty:
        raise SystemExit("Nenhum dado retornado pelo Yahoo Finance para o período.")

    ohlc = (
      df_yf.reset_index()[["Date", "Open", "High", "Low", "Close"]]
        .rename(columns={
          "Date": "data",
          "Open": "abertura",
          "High": "alta",
          "Low": "baixa",
          "Close": "fechamento",
        })
    )
    ohlc[["abertura","fechamento","alta","baixa"]] = (
      ohlc[["abertura","fechamento","alta","baixa"]].round(4)
    )

    min_ohlc = pd.to_datetime(ohlc["data"]).min().date()
    max_ohlc = pd.to_datetime(ohlc["data"]).max().date()
    print(f"[{now()}] [YF] Linhas OHLC: {len(ohlc)} | Janela: {min_ohlc} → {max_ohlc}")

    s.ok()
except Exception as e:
    s.fail(e)
    raise

[2025-10-25 20:01:07] [Yahoo OHLC - Coleta] INÍCIO
[2025-10-25 20:01:08] [YF] Linhas OHLC: 147 | Janela: 2025-04-01 → 2025-10-24
[2025-10-25 20:01:08] [Yahoo OHLC - Coleta] FIM — OK (0.37s)


In [13]:
# =============================================================================
# 4) Yahoo OHLC — carga no DuckDB
# =============================================================================
s = Step("Yahoo OHLC - Carga DuckDB")
try:
    DB_PATH = "dados_dolar.duckdb"
    TABELA = "dolar_ohlc"
    con = duckdb.connect(DB_PATH)

    con.execute(f"""
    CREATE TABLE IF NOT EXISTS {TABELA} (
     data DATE PRIMARY KEY,
     abertura  DOUBLE,
     fechamento DOUBLE,
     alta    DOUBLE,
     baixa   DOUBLE
    );
    """)

    # 1) Achatar colunas vindas do yfinance (MultiIndex -> string simples)
    if isinstance(ohlc.columns, pd.MultiIndex):
        ohlc.columns = ohlc.columns.get_level_values(0)

    # 2) Garantir tipos
    ohlc["data"] = pd.to_datetime(ohlc["data"]).dt.date  # vira datetime.date
    for col in ["abertura", "fechamento", "alta", "baixa"]:
        ohlc[col] = pd.to_numeric(ohlc[col], errors="coerce")

    print("Colunas pandas:", list(ohlc.columns))  # ['data','abertura','fechamento','alta','baixa']

    con.register("df_stage", ohlc)
    # UPSERT simples usando a PK(data)
    
    con.execute(f"""
    INSERT OR REPLACE INTO {TABELA}
    SELECT
    CAST("data" AS DATE)        AS data,
    CAST("abertura" AS DOUBLE)  AS abertura,
    CAST("fechamento" AS DOUBLE)AS fechamento,
    CAST("alta" AS DOUBLE)      AS alta,
    CAST("baixa" AS DOUBLE)     AS baixa
    FROM df_stage;
    """)

    total_ohlc_db = con.execute(f"SELECT COUNT(*) FROM {TABELA};").fetchone()[0]
    print(f"[{now()}] [YF] Linhas na tabela {TABELA}: {total_ohlc_db}")

    con.execute("""
    COPY dolar_ohlc TO './exports/dolar_ohlc.csv' (HEADER, DELIMITER ',');
    """)
    print(f"[{now()}] [YF] Export CSV -> ./exports/dolar_ohlc.csv")

    con.unregister("df_stage")
    con.close()

    # também exporta o DF direto (útil pra auditoria)
    ohlc.to_csv('./exports/dolar_ohlc_yahoo.csv', index=False)
    print(f"[{now()}] [YF] Export CSV (via pandas) -> ./exports/dolar_ohlc_yahoo.csv")

    s.ok("Carga concluída")
except Exception as e:
    s.fail(e)
    raise

[2025-10-25 20:01:08] [Yahoo OHLC - Carga DuckDB] INÍCIO
Colunas pandas: ['data', 'abertura', 'alta', 'baixa', 'fechamento']
[2025-10-25 20:01:08] [YF] Linhas na tabela dolar_ohlc: 147
[2025-10-25 20:01:08] [YF] Export CSV -> ./exports/dolar_ohlc.csv
[2025-10-25 20:01:08] [YF] Export CSV (via pandas) -> ./exports/dolar_ohlc_yahoo.csv
[2025-10-25 20:01:08] [Yahoo OHLC - Carga DuckDB] FIM — Carga concluída (0.06s)


In [14]:
# =============================================================================
# 5) Resumo final (logs de status + contagens)
# =============================================================================
print("\nResumo – Séries (PTAX & Yahoo)")
print("--------------------------------")
print(f"[{now()}] PTAX diário: {len(df_daily)} | Janela: {min_ptax} → {max_ptax}")
print(f"[{now()}] OHLC linhas: {len(ohlc)} | Janela: {min_ohlc} → {max_ohlc}")


Resumo – Séries (PTAX & Yahoo)
--------------------------------
[2025-10-25 20:01:08] PTAX diário: 145 | Janela: 2025-04-01 → 2025-10-24
[2025-10-25 20:01:08] OHLC linhas: 147 | Janela: 2025-04-01 → 2025-10-24
